In [ ]:
import lightgbm as lgb
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import clone
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import linear_model
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings("ignore")

In [ ]:
def pre_setting():
    last_train_data_add = pd.read_csv("last_train.csv")
    last_test_data_add = pd.read_csv("last_test.csv")
    
    X = last_train_data_add
    y = last_test_data_add
    
#     clf1 = linear_model.LogisticRegression(n_jobs=-1)
#     clf2 = RandomForestClassifier(n_jobs=-1)
#     clf3 = ExtraTreesClassifier(n_jobs=-1)
#     clf4 = xgb.XGBClassifier(nthread=3, n_jobs=-1)
    clf5 = lgb.LGBMClassifier(nthread=3, n_jobs=-1, reg_alpha=1)
#     eclf2 = VotingClassifier(estimators=[('log', clf1), ('rf', clf2), ('ex', clf3), ('xgb', clf4), ('lgb', clf5)], voting='soft', weights=[1,1,1,1,1])
    
    return X, y, clf5

X, y, clf5  = pre_setting()

In [ ]:
def submit_kaggle(df_train, df_test, model, user_id, target):
    
    le = LabelEncoder()

    y_train = le.fit_transform(df_train[target])
    X_train = df_train.drop([target, user_id], axis = 1)
    
    X_test_id = df_test[user_id]
    X_test = df_test.drop([target, user_id], axis = 1)
    

    print("model fitting 시작")
    
    
    model1 = model
    parameters = {'learning_rate':[0.1,0.075], 'n_estimators':[75, 125], 'num_leaves':[31,50], 'subsample':[1,0.75] }
    clf = GridSearchCV(n_jobs=-1, estimator=model1, param_grid=parameters).fit(X_train, y_train)
    
    
    predic_proba = clf.predict_proba(X_test)
    
    clf.best_score_()
    
    print("model fitting 종료")

    df_submit = pd.DataFrame(columns=["id", "country"])
    ids = []
    cts = []
    for i in range(len(X_test_id)):
        idx = X_test_id.iloc[i]
        ids += [idx] * 5
        cts += le.inverse_transform(np.argsort(predic_proba[i])[::-1])[:5].tolist()
    df_submit["id"] = ids
    df_submit["country"] = cts
    df_submit.to_csv('submit_xg.csv', index = False)
    print("csv file 생성")
    !kaggle competitions submit -c airbnb-recruiting-new-user-bookings -f 'submit_xg.csv' -m "Message"

In [ ]:
submit_kaggle(X, y, clf5, "id", "country_destination")

## boosting type
#gbtree = 0.88388, 0.87941
#dart = 0.88282, 0.87850
#rf = error
#goss = 0.87924, 0.87495

## reg alpha, lambda
#1, 0 = 0.88421, 0.87880
#0.5, 0.5 = 0.88409, 0.87876
#0.75, 0.25 = 0.88367, 0.87836

## n_estimators
#200 = 0.88418, 0.87937
#300 = 0.88412, 0.87875
#1000 = 0.88210, 0.87872

### num_leaves
#50 = 0.88384, 0.87988

### learning rate
#0.2 = 0.88339, 0.87920
